In [30]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [18]:
#set up executable path & the url for scraping
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [/Users/kimberlybraud/.wdm/drivers/chromedriver/mac64/92.0.4515.107/chromedriver] found in cache


In [19]:
#assign the url and instruct the browser to visit it.
url = 'https://redplanetscience.com'
browser.visit(url)


In [20]:
# Optional delay of 1 second before searching for components
#The optional delay is useful because sometimes dynamic pages take a little while to load,
#especially if they are image-heavy.
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [21]:
# set up HTML parser

html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

#slide_elem as the variable to look for the <div /> tag and its descendent (the other tags within the <div /> element). 
#This is our parent element
#The . is used for selecting classes, such as list_text, so the code 'div.list_text' pinpoints the <div /> tag with the class of list_text

In [22]:
#assign the title and summary text to variables we'll reference later
slide_elem.find('div', class_='content_title')

<div class="content_title">What's Mars Solar Conjunction, and Why Does It Matter?</div>

In [23]:
#get just the text - not the extra HTML stuff
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"What's Mars Solar Conjunction, and Why Does It Matter?"

In [24]:
#get just the article teaser text
news_p = slide_elem.find('div', class_= 'article_teaser_body').get_text()
news_p

'NASA spacecraft at Mars are going to be on their own for a few weeks when the Sun comes between Mars and Earth, interrupting communications.'

### Featured Images

In [25]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [26]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [27]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [28]:
#use the image tag and class (<img />and fancybox-img) 
#to build the URL to the full-size image.

# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars3.jpg'

In [29]:
# add the base URL to our code. / Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

In [33]:
#scrape the entire table with pandas' .read_html()
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns = ['description', 'Mars', 'Earth']
df.set_index('description', inplace = True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [34]:
# convert df back to html-ready code
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [35]:
# end the automated browsing session
browser.quit()